# Options Portfolio Optimization

## 1. Data collection

We import required libraries and modules and collect data for companies of the S&P 500.

In [1]:
import numpy as np
import pandas as pd
import time
import yfinance as yf
import matplotlib.pyplot as plt
import functions as f
import pulp


%load_ext autoreload
%autoreload 2

In [2]:
companies = f.get_companies()

We filter the tickers according to market cap market volume and desired dates. 

In [3]:
filtered_companies = f.filter_all_companies(companies=companies)

In [4]:
last_row_df = filtered_companies.tail(1).T
last_row_df.reset_index(inplace=True)
last_row_df.columns = ['Company', 'Last Value']
ticker_prices = last_row_df.set_index('Company')['Last Value'].round(2).to_dict()
last_row_df.set_index('Company', inplace=True)

We get the calls, puts and spot price for each ticker.

In [5]:
#tickers = filtered_companies.columns

tickers = ['AMD', 'GOOGL', 'GOOG', 'AMZN', 'AAPL', 'T', 'BAC', 'BA', 'BMY', 'AVGO',
       'CMG', 'CSCO', 'C', 'KO', 'CMCSA', 'CSX', 'CVS', 'XOM', 'FCX', 'GM',
       'INTC', 'KMI', 'LRCX', 'META', 'MU', 'MSFT', 'NKE', 'NVDA', 'PLTR',
       'PYPL', 'PFE', 'PCG', 'SLB', 'TSLA', 'UBER', 'VZ', 'WMT', 'WFC']

option_chains = {}

for ticker in tickers:
    try:
        # Call the function for each ticker
        calls_all, puts_all, spot_price = f.get_option_chains_spot(ticker_symbol=ticker)
        
        # Store the result in the dictionary
        option_chains[ticker] = {
            'calls': calls_all,
            'puts': puts_all,
            'spot_price': spot_price
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

Attempt 1 failed with error: Expecting value: line 1 column 1 (char 0) - Retrying after 2 seconds...


$NKE: possibly delisted; no price data found  (period=1d)


Attempt 2 failed with error: No historical data available for ticker NKE. - Retrying after 2 seconds...


$NKE: possibly delisted; no price data found  (period=1d)


Attempt 3 failed with error: No historical data available for ticker NKE. - Retrying after 2 seconds...
Error fetching data for NKE: Failed to get spot price and options data for ticker NKE after 3 attempts.


...

Attempt 1 failed with error: No historical data available for ticker SLB. - Retrying after 2 seconds...


$SLB: possibly delisted; no price data found  (period=1d)


Attempt 2 failed with error: No historical data available for ticker SLB. - Retrying after 2 seconds...


$SLB: possibly delisted; no price data found  (period=1d)


Attempt 3 failed with error: No historical data available for ticker SLB. - Retrying after 2 seconds...
Error fetching data for SLB: Failed to get spot price and options data for ticker SLB after 3 attempts.


In [6]:
#option_chains

| **Option Type**  | **Underlying Price vs. Strike Price** | **Moneyness**          |
|-------------------|---------------------------------------|------------------------|
| **Call Option**   | Asset Price > Strike Price            | In the Money (ITM)     |
|                   | Asset Price = Strike Price            | At the Money (ATM)     |
|                   | Asset Price < Strike Price            | Out of the Money (OTM) |
| **Put Option**    | Asset Price < Strike Price            | In the Money (ITM)     |
|                   | Asset Price = Strike Price            | At the Money (ATM)     |
|                   | Asset Price > Strike Price            | Out of the Money (OTM) |


We filter the options to exclude irregular volatilities and maturity dates close to present date.

In [7]:
filtered_options = {}

r = 0.03
lower_bound = 0.0821  
upper_bound = 2    
tolerance = 1      
min_iv = 0.05         
max_iv = 1.5         

for ticker, data in option_chains.items():
    try:
        spot_price = data['spot_price']  # Get the spot price for the ticker

        for mode in ['calls', 'puts']:  # Iterate through calls and puts
            options_df = data[mode]  # Get the DataFrame for this mode

            # Remove options with missing or zero implied volatility
            options_df = options_df[options_df['impliedVolatility'] > 0]
            options_df = options_df[options_df['impliedVolatility'].notnull()]

            # Ensure 'expiration' is in string format 'YYYY-MM-DD'
            options_df['expiration'] = options_df['expiration'].astype(str)

            # Calculate 'time_to_expiration' using the corrected function
            options_df['time_to_expiration'] = options_df['expiration'].apply(f.calculate_time_to_expiration_from_today)

            # Apply filtering criteria
            filtered = options_df[
                (options_df["time_to_expiration"] >= lower_bound) &
                (options_df["time_to_expiration"] <= upper_bound) &
                (options_df["strike"] >= spot_price * (1 - tolerance)) &
                (options_df["strike"] <= spot_price * (1 + tolerance)) &
                (options_df["impliedVolatility"] >= min_iv) &
                (options_df["impliedVolatility"] <= max_iv)
            ]

            # Ensure the ticker exists in the filtered options dictionary
            if ticker not in filtered_options:
                filtered_options[ticker] = {"spot_price": spot_price}

            # Store the filtered options
            filtered_options[ticker][mode] = filtered

            # Print a summary for verification
            print(f"Filtered {len(filtered)} {mode} for {ticker}.")
    except Exception as e:
        print(f"Error processing {ticker} {mode}: {e}")

Filtered 466 calls for AMD.
Filtered 461 puts for AMD.
Filtered 533 calls for GOOGL.
Filtered 437 puts for GOOGL.
Filtered 342 calls for GOOG.
Filtered 300 puts for GOOG.
Filtered 497 calls for AMZN.
Filtered 464 puts for AMZN.
Filtered 588 calls for AAPL.
Filtered 563 puts for AAPL.
Filtered 103 calls for T.
Filtered 105 puts for T.
Filtered 239 calls for BAC.
Filtered 236 puts for BAC.
Filtered 422 calls for BA.
Filtered 359 puts for BA.
Filtered 174 calls for BMY.
Filtered 175 puts for BMY.
Filtered 774 calls for AVGO.
Filtered 861 puts for AVGO.
Filtered 519 calls for CMG.
Filtered 490 puts for CMG.
Filtered 164 calls for CSCO.
Filtered 165 puts for CSCO.
Filtered 205 calls for C.
Filtered 205 puts for C.
Filtered 146 calls for KO.
Filtered 129 puts for KO.
Filtered 88 calls for CMCSA.
Filtered 86 puts for CMCSA.
Filtered 70 calls for CSX.
Filtered 68 puts for CSX.
Filtered 200 calls for CVS.
Filtered 188 puts for CVS.
Filtered 263 calls for XOM.
Filtered 229 puts for XOM.
Filtered

In [8]:
#filtered_options

## 2. Black-Scholes calculation

We calculate the Black-Scholes price for each of the options.

In [9]:
for ticker in filtered_options.keys():
    # Round the spot price to 2 decimal places and update it in the dictionary
    filtered_options[ticker]["spot_price"] = round(filtered_options[ticker]["spot_price"], 2)
    spot = filtered_options[ticker]["spot_price"]

    # Vectorized calculations for "calls"
    if "calls" in filtered_options[ticker]:
        calls = filtered_options[ticker]["calls"].copy()  # Make a copy of the DataFrame
        calls["blackScholes_Price"] = f.black_scholes_call(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        # Round Black-Scholes Price to 2 decimals
        calls["blackScholes_Price"] = calls["blackScholes_Price"].round(2)

        # Delta, Vega, Gamma, Theta, Rho Calculations (unchanged)
        calls["delta"] = f.delta_call(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        calls["vega"] = f.vega(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        calls["gamma"] = f.gamma(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        calls["theta"] = f.theta_call(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        calls["rho"] = f.rho_call(
            S=spot,
            K=calls["strike"],
            T=calls["time_to_expiration"],
            r=r,
            sigma=calls["impliedVolatility"]
        )
        # Save the updated DataFrame back
        filtered_options[ticker]["calls"] = calls

    # Vectorized calculations for "puts"
    if "puts" in filtered_options[ticker]:
        puts = filtered_options[ticker]["puts"].copy()  # Make a copy of the DataFrame
        puts["blackScholes_Price"] = f.black_scholes_put(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        # Round Black-Scholes Price to 2 decimals
        puts["blackScholes_Price"] = puts["blackScholes_Price"].round(2)

        # Delta, Vega, Gamma, Theta, Rho Calculations (unchanged)
        puts["delta"] = f.delta_put(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        puts["vega"] = f.vega(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        puts["gamma"] = f.gamma(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        puts["theta"] = f.theta_put(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        puts["rho"] = f.rho_put(
            S=spot,
            K=puts["strike"],
            T=puts["time_to_expiration"],
            r=r,
            sigma=puts["impliedVolatility"]
        )
        # Save the updated DataFrame back
        filtered_options[ticker]["puts"] = puts

In [10]:
#filtered_options

We combine the calls and puts into one dataframe and drop and possible NaN values.

In [11]:
# Combine calls and puts across all stocks
all_options = []
for stock, data in filtered_options.items():
    spot_price = data['spot_price']  # Get the spot price of the stock
    
    if not data['calls'].empty:
        # Add stock and spot price columns to calls
        data['calls']['type'] = 'call'
        data['calls']['stock'] = stock
        data['calls']['spot_price'] = spot_price
        all_options.append(data['calls'])
    
    if not data['puts'].empty:
        # Add stock and spot price columns to puts
        data['puts']['type'] = 'put'
        data['puts']['stock'] = stock
        data['puts']['spot_price'] = spot_price
        all_options.append(data['puts'])

# Concatenate into a single DataFrame
all_options_df = pd.concat(all_options, ignore_index=True)

# Find rows where all columns are NaN
nan_rows = all_options_df[all_options_df.isnull().all(axis=1)]
print(f"Rows with all NaN values:\n{nan_rows}")

# Find rows where any column is NaN
partial_nan_rows = all_options_df[all_options_df.isnull().any(axis=1)]
print(f"Rows with partial NaN values:\n{partial_nan_rows}")

# Drop rows where all columns are NaN
all_options_df = all_options_df.dropna(how='all')

# Define key columns that must not have NaN values
key_columns = ['strike', 'blackScholes_Price', 'delta']

# Drop rows where any key column is NaN
all_options_df = all_options_df.dropna(subset=key_columns)

# Check for remaining NaN values
print(all_options_df.isnull().sum())

# Check the shape of the cleaned DataFrame
print(f"Number of rows after cleaning: {len(all_options_df)}")

Rows with all NaN values:
Empty DataFrame
Columns: [strike, lastPrice, impliedVolatility, expiration, time_to_expiration, blackScholes_Price, delta, vega, gamma, theta, rho, type, stock, spot_price]
Index: []
Rows with partial NaN values:
Empty DataFrame
Columns: [strike, lastPrice, impliedVolatility, expiration, time_to_expiration, blackScholes_Price, delta, vega, gamma, theta, rho, type, stock, spot_price]
Index: []
strike                0
lastPrice             0
impliedVolatility     0
expiration            0
time_to_expiration    0
blackScholes_Price    0
delta                 0
vega                  0
gamma                 0
theta                 0
rho                   0
type                  0
stock                 0
spot_price            0
dtype: int64
Number of rows after cleaning: 23123


In [12]:
required_columns = ['strike', 'spot_price', 'blackScholes_Price', 'delta', 'gamma', 'vega', 'theta', 'type']
missing_columns = [col for col in required_columns if col not in all_options_df.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")


## 3. Optimization

We initialize the weights for our optimization problem

In [13]:
all_options_df['initial_weight'] = 1 / len(all_options_df)

In [14]:
#all_options_df

In [15]:
bounds = [(0, 1) for _ in range(len(all_options_df))]

We assume that our expected returns is the Black-Scholes price of each of the options minus the real market price of the option, thus assuming that the option is overvalued is the expected return is negative and vice-versa.

In [16]:
all_options_df['market_price'] = all_options_df['lastPrice']
all_options_df['market_price'] = all_options_df['market_price'].astype(float)
all_options_df['delta'] = all_options_df['delta'].astype(float)
all_options_df['vega'] = all_options_df['vega'].astype(float)
all_options_df['gamma'] = all_options_df['gamma'].astype(float)
all_options_df['theta'] = all_options_df['theta'].astype(float)
all_options_df['rho'] = all_options_df['rho'].astype(float)
all_options_df['expected_return'] = all_options_df['blackScholes_Price'] - all_options_df['market_price']

For our problem, we assume a starting budget of $100,000 and fix the total greeks of the portfolio as constraints to the optimization problem. Our objective function is to maximize our expected returns. We use PuLP as our optimizer.

In [17]:
budget = 100000  # Your budget in dollars
total_option_vega = all_options_df['vega'].sum()

# Define Greek limits (adjust these values based on your risk tolerance)
gamma_min = 0  # Positive Gamma
gamma_max = 0.05 * budget  # No upper limit unless specified

vega_min = 0.2 * total_option_vega  # 20% of total option Vega
vega_max = 0.5 * total_option_vega  # 50% of total option Vega

theta_min = -0.01 * budget  # Allowable Theta loss per day (adjust as needed)
theta_max = 0  # Typically, Theta is negative for options buyers

rho_min = None  # Set if you have a specific Rho target
rho_max = None  # Set if you have a specific Rho limit

# Optimization Setup
prob = pulp.LpProblem("OptionsPortfolioOptimization", pulp.LpMaximize)

# Decision Variables
N = [pulp.LpVariable(f"N_{i}", lowBound=0) for i in range(len(all_options_df))]

# Objective Function: Maximize total portfolio Vega
prob += pulp.lpSum([N[i] * all_options_df['vega'].iloc[i] * 100 for i in range(len(all_options_df))])

# Budget Constraint
prob += pulp.lpSum([N[i] * all_options_df['market_price'].iloc[i] * 100 for i in range(len(all_options_df))]) <= budget

# Delta-neutral Constraint
prob += pulp.lpSum([N[i] * all_options_df['delta'].iloc[i] * 100 for i in range(len(all_options_df))]) == 0

# Gamma Constraint
prob += pulp.lpSum([N[i] * all_options_df['gamma'].iloc[i] * 100 for i in range(len(all_options_df))]) >= gamma_min
# If you want to set an upper limit, uncomment the following line and set gamma_max
# prob += pulp.lpSum([N[i] * all_options_df['gamma'].iloc[i] * 100 for i in range(len(all_options_df))]) <= gamma_max

# Vega Constraint: Total portfolio Vega should be between vega_min and vega_max
prob += pulp.lpSum([N[i] * all_options_df['vega'].iloc[i] * 100 for i in range(len(all_options_df))]) >= vega_min
prob += pulp.lpSum([N[i] * all_options_df['vega'].iloc[i] * 100 for i in range(len(all_options_df))]) <= vega_max

# Theta Constraint
prob += pulp.lpSum([N[i] * all_options_df['theta'].iloc[i] * 100 for i in range(len(all_options_df))]) >= theta_min
prob += pulp.lpSum([N[i] * all_options_df['theta'].iloc[i] * 100 for i in range(len(all_options_df))]) <= theta_max

# Rho Constraints (if needed)
# If you have specific Rho limits, set rho_min and rho_max and uncomment the following lines
# prob += pulp.lpSum([N[i] * all_options_df['rho'].iloc[i] * 100 for i in range(len(all_options_df))]) >= rho_min
# prob += pulp.lpSum([N[i] * all_options_df['rho'].iloc[i] * 100 for i in range(len(all_options_df))]) <= rho_max

# Solve the Problem
prob.solve()

# Solution Status
print(f"Status: {pulp.LpStatus[prob.status]}")

# Extract Optimized Quantities
optimized_N = [var.varValue for var in N]
all_options_df['optimized_N'] = optimized_N

# Optimized Portfolio
optimized_portfolio = all_options_df[all_options_df['optimized_N'] > 0]
print(optimized_portfolio[['strike', 'type', 'stock', 'market_price', 'delta', 'vega', 'gamma', 'theta', 'rho', 'expected_return', 'optimized_N']])

# Portfolio Metrics
total_delta = (optimized_portfolio['delta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_vega = (optimized_portfolio['vega'] * optimized_portfolio['optimized_N'] * 100).sum()
total_gamma = (optimized_portfolio['gamma'] * optimized_portfolio['optimized_N'] * 100).sum()
total_theta = (optimized_portfolio['theta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_rho = (optimized_portfolio['rho'] * optimized_portfolio['optimized_N'] * 100).sum()
total_cost = (optimized_portfolio['market_price'] * optimized_portfolio['optimized_N'] * 100).sum()
expected_return = (optimized_portfolio['expected_return'] * optimized_portfolio['optimized_N'] * 100).sum()

print(" ")
print(f"Total Portfolio Delta: {total_delta}")
print(f"Total Portfolio Vega: {total_vega}")
print(f"Total Portfolio Gamma: {total_gamma}")
print(f"Total Portfolio Theta: {total_theta}")
print(f"Total Portfolio Rho: {total_rho}")
print(f"Total Portfolio Cost: ${total_cost:.2f}")
print(f"Portfolio expected return: ${expected_return:.2f}")



Status: Optimal
       strike  type stock  market_price     delta        vega     gamma  \
4748     22.0  call     T          2.90  0.681405    8.857743  0.061629   
18384   520.0   put  MSFT         68.45 -0.998984    0.549306  0.000118   
19047   440.0   put  MSFT         46.20 -0.464868  207.322251  0.004074   

           theta         rho  expected_return  optimized_N  
4748   -1.276799   14.318105             0.40    16.452563  
18384  15.186346  -76.961730            32.24     5.250409  
19047  -5.355448 -366.622676            -5.82    12.833270  
 
Total Portfolio Delta: -4.526913699010038e-06
Total Portfolio Vega: 280923.9086444495
Total Portfolio Gamma: 106.68530444423327
Total Portfolio Theta: -999.9998284674339
Total Portfolio Rho: -487347.881732918
Total Portfolio Cost: $100000.00
Portfolio expected return: $10116.46


We intentionally calibrated the optimizer to return float weights instead of integers, because integer calculation for such a portfolio was problematic. Below we will process our float weights into integers an re-check our constraints. In this case, the process failed and so we will re-calibrate our outputs again, until we pass the constraint criteria, or at least violate them in acceptable margins.

In [18]:
# Round optimized_N to integers
all_options_df['optimized_N'] = all_options_df['optimized_N'].apply(lambda x: round(x) if x > 0 else 0)

# Recalculate Portfolio Metrics with Rounded Quantities
optimized_portfolio = all_options_df[all_options_df['optimized_N'] > 0]

total_delta = (optimized_portfolio['delta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_vega = (optimized_portfolio['vega'] * optimized_portfolio['optimized_N'] * 100).sum()
total_gamma = (optimized_portfolio['gamma'] * optimized_portfolio['optimized_N'] * 100).sum()
total_theta = (optimized_portfolio['theta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_rho = (optimized_portfolio['rho'] * optimized_portfolio['optimized_N'] * 100).sum()
total_cost = (optimized_portfolio['market_price'] * optimized_portfolio['optimized_N'] * 100).sum()
expected_return = (optimized_portfolio['expected_return'] * optimized_portfolio['optimized_N'] * 100).sum()


# Print the updated rounded portfolio
print(optimized_portfolio[['strike', 'type', 'stock', 'market_price', 'delta', 'vega', 'gamma', 'theta', 'rho','expected_return', 'optimized_N']])

# Print updated portfolio metrics
print(" ")
print(f"Total Portfolio Delta (Rounded): {total_delta}")
print(f"Total Portfolio Vega (Rounded): {total_vega}")
print(f"Total Portfolio Gamma (Rounded): {total_gamma}")
print(f"Total Portfolio Theta (Rounded): {total_theta}")
print(f"Total Portfolio Rho (Rounded): {total_rho}")
print(f"Total Portfolio Cost (Rounded): ${total_cost:.2f}")
print(f"Portfolio expected return: ${expected_return:.2f}")


# Check if any constraints are violated
print("\nConstraint Validation After Rounding:")
print(f"Delta Neutral: {'PASS' if abs(total_delta) < 1e-5 else 'FAIL'}")
print(f"Vega within limits: {'PASS' if vega_min <= total_vega <= vega_max else 'FAIL'}")
print(f"Gamma >= gamma_min: {'PASS' if total_gamma >= gamma_min else 'FAIL'}")
print(f"Theta within limits: {'PASS' if theta_min <= total_theta <= theta_max else 'FAIL'}")
print(f"Budget constraint: {'PASS' if total_cost <= budget else 'FAIL'}")


       strike  type stock  market_price     delta        vega     gamma  \
4748     22.0  call     T          2.90  0.681405    8.857743  0.061629   
18384   520.0   put  MSFT         68.45 -0.998984    0.549306  0.000118   
19047   440.0   put  MSFT         46.20 -0.464868  207.322251  0.004074   

           theta         rho  expected_return  optimized_N  
4748   -1.276799   14.318105             0.40           16  
18384  15.186346  -76.961730            32.24            5  
19047  -5.355448 -366.622676            -5.82           13  
 
Total Portfolio Delta (Rounded): -13.573162675733101
Total Portfolio Vega (Rounded): 283965.96876427135
Total Portfolio Gamma (Rounded): 103.96119123936695
Total Portfolio Theta (Rounded): -1411.7874985136596
Total Portfolio Rho (Rounded): -492181.37660198525
Total Portfolio Cost (Rounded): $98925.00
Portfolio expected return: $9194.00

Constraint Validation After Rounding:
Delta Neutral: FAIL
Vega within limits: PASS
Gamma >= gamma_min: PASS
Theta 

In [19]:
optimized_portfolio = f.adjust_weights_to_constraints(
    optimized_portfolio, budget, vega_min, vega_max, theta_min, theta_max
)

# Recalculate and validate metrics
total_delta = (optimized_portfolio['delta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_vega = (optimized_portfolio['vega'] * optimized_portfolio['optimized_N'] * 100).sum()
total_gamma = (optimized_portfolio['gamma'] * optimized_portfolio['optimized_N'] * 100).sum()
total_theta = (optimized_portfolio['theta'] * optimized_portfolio['optimized_N'] * 100).sum()
total_rho = (optimized_portfolio['rho'] * optimized_portfolio['optimized_N'] * 100).sum()
total_cost = (optimized_portfolio['market_price'] * optimized_portfolio['optimized_N'] * 100).sum()
expected_return = (optimized_portfolio['expected_return'] * optimized_portfolio['optimized_N'] * 100).sum()


# Print adjusted portfolio and metrics
print(optimized_portfolio[['strike', 'type', 'stock', 'market_price', 'delta', 'vega', 'gamma', 'theta', 'rho','expected_return', 'optimized_N']])

print(" ")
print(f"Total Portfolio Delta (Adjusted): {total_delta}")
print(f"Total Portfolio Vega (Adjusted): {total_vega}")
print(f"Total Portfolio Gamma (Adjusted): {total_gamma}")
print(f"Total Portfolio Theta (Adjusted): {total_theta}")
print(f"Total Portfolio Rho (Adjusted): {total_rho}")
print(f"Total Portfolio Cost (Adjusted): ${total_cost:.2f}")
print(f"Portfolio expected return: ${expected_return:.2f}")

print("\nConstraint Validation After adjusting:")
print(f"Delta Neutral: {'PASS' if abs(total_delta) < 1e-5 else 'FAIL'}")
print(f"Vega within limits: {'PASS' if vega_min <= total_vega <= vega_max else 'FAIL'}")
print(f"Gamma >= gamma_min: {'PASS' if total_gamma >= gamma_min else 'FAIL'}")
print(f"Theta within limits: {'PASS' if theta_min <= total_theta <= theta_max else 'FAIL'}")
print(f"Budget constraint: {'PASS' if total_cost <= budget else 'FAIL'}")


       strike  type stock  market_price     delta        vega     gamma  \
4748     22.0  call     T          2.90  0.681405    8.857743  0.061629   
18384   520.0   put  MSFT         68.45 -0.998984    0.549306  0.000118   
19047   440.0   put  MSFT         46.20 -0.464868  207.322251  0.004074   

           theta         rho  expected_return  optimized_N  
4748   -1.276799   14.318105             0.40           13  
18384  15.186346  -76.961730            32.24            5  
19047  -5.355448 -366.622676            -5.82           13  
 
Total Portfolio Delta (Adjusted): -217.9946087575642
Total Portfolio Vega (Adjusted): 281308.64581731963
Total Portfolio Gamma (Adjusted): 85.47261160892822
Total Portfolio Theta (Adjusted): -1028.7479215093826
Total Portfolio Rho (Adjusted): -496476.8080107872
Total Portfolio Cost (Adjusted): $98055.00
Portfolio expected return: $9074.00

Constraint Validation After adjusting:
Delta Neutral: FAIL
Vega within limits: PASS
Gamma >= gamma_min: PASS
Th

We notice above that essentially the optimizer used only three total types of options for our portfolio, our greek constraints are inside the intervals proposed in the beginning, except delta, which is relatively close to 0. The expected returns for this portfolio is: $3921.00

In [20]:
optimized_portfolio

,strike,lastPrice,impliedVolatility,expiration,time_to_expiration,blackScholes_Price,delta,vega,gamma,theta,rho,type,stock,spot_price,initial_weight,market_price,expected_return,optimized_N
4748,22.0,2.90,0.233406,2026-01-16,1.146045,3.30,0.681405,8.857743,0.061629,-1.276799,14.318105,call,T,23.18,0.000043,2.90,0.40,13
18384,520.0,68.45,0.179696,2025-01-17,0.148785,100.69,-0.998984,0.549306,0.000118,15.186346,-76.961730,put,MSFT,417.00,0.000043,68.45,32.24,5
19047,440.0,46.20,0.186966,2026-06-18,1.565223,40.38,-0.464868,207.322251,0.004074,-5.355448,-366.622676,put,MSFT,417.00,0.000043,46.20,-5.82,13
